<a href="https://colab.research.google.com/github/pedr0-almeida/AI2023/blob/main/Tarefa_1_Trocadores_de_Calor_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from random import randint

In [ ]:
def CalcFit(fun):
    result=0
    if(fun>=0):
      result=1/(fun+1)
    else:
      result=1+abs(fun)
    return result

In [ ]:
# Target: temperatures initial and target
Target = {'H1':[180,75],'H2':[280,120],'H3':[180,75],'H4':[140,40],
          'H5':[220,120],'H6':[180,55],'H7':[200,60],'H8':[120,40],
          'C1':[40,230],'C2':[100,220],'C3':[40,190],'C4':[50,190],
          'C5':[50,250],'C6':[90,190],'C7':[160,250]}

Util = {'cold':[25,40],'hot':[325,325]}

# Temperatures (hot streams first, cold after and starting with H/C for hot and cold) otherwise need to adapt the code
Prop = {'H1':[30,2],'H2':[60,1],'H3':[30,2],'H4':[30,1],'H5':[50,1],
        'H6':[35,2],'H7':[30,0.4],'H8':[100,0.5],'C1':[20,1],'C2':[60,1],
        'C3':[35,2],'C4':[30,2],'C5':[60,2],'C6':[50,1],'C7':[60,3],
        'cold':[4.186,2],'hot':[431.379,1]} # CP e H

n_hot = 8

# w input of swarm intelligence size = n_exchages*num_hot
n_trocas = 3


In [ ]:
def Cost_find(w,Target = Target, Prop = Prop, Util= Util, n_trocas = n_trocas, ans = False ):
  w = list(w)
  Gasta_cold = [] # [Area,flow,kwy] for each stream
  Gasta_hot = []  # [Area,flow,kwy] for each stream
  U_q =[]
  names = list(Target.keys())
  n_hot, n_cold, stream= 0, 0, ['No']
  # constroi temperaturas
  Temp ={}
  for word in names:
    Temp[word]=[Target[word][0]]

  for word in names:
    if 'H' in word:
      n_hot += 1
    elif 'C' in word:
      n_cold += 1
      stream.append(word)
  
  N = len(stream)
  r = 1/N
  trocas =[]
  Areas = []
  n = n_trocas*n_hot
  
  x = w[0:n]
  tx = w[n:(2*n)]

  cont = 0
  for xi in x:
    total = 0
    cont += 1
    for k in range(N):
      total += r
      if xi <= total:
        trocas.append(stream[k])
        break 
  n1, n2, matrix_trocas, xt_hot = 0, 0, [],[]

  while(len(trocas) < n):
    trocas.append('No')
  
  num = int(len(trocas)/n_hot)
  # print('num',num, trocas)

  for idx in range(n_hot):
    tr, dr =  [], []
    n2 = num*(1+idx)
    tr.append(trocas[n1:n2])
    dr.append(tx[n1:n2])
    matrix_trocas.append(tr[0])
    xt_hot.append(dr[0])
    n1 = n2
  
  idx = 0
  # update hot temperature
  for j in range(n_trocas):
    for k in range(n_hot):
      x = Temp[names[k]][idx]
      # print('matrix_trocas, k,j',matrix_trocas, k,j)
      cold = matrix_trocas[k][j]
      hot = names[k]
      if cold == 'No':
        Temp[names[k]].append(x)
      else:
        # x, y = hot and cold temperatures
        dx = Target[hot][0]-Target[hot][1]
        x0 = x* 1.0
        y0 = Temp[cold][-1]
        x = x - dx * xt_hot[k][j]
        CP_c = Prop[cold][0]
        CP_h = Prop[hot][0]
        Q = CP_c* dx * xt_hot[k][j]
        y = CP_h*dx * xt_hot[k][j]/CP_c + Temp[cold][-1]
        d1 = x0- y
        d2 = x - y0
        U = 1.0 / ( 1.0/Prop[cold][1]+ 1.0/Prop[hot][1])
        if d2 != 0 and d1*d2 > 0:
          check = 'ok'
        else:
          check = 'no'
        if x0 > y and x > y0 and check == 'ok': # so troca calor se a temperatura fria final for menor q a temperatura quente final
          Temp[cold].append(y)
          Temp[names[k]].append(x)
          LMDT = (d1 - d2) / np.log(d1/d2)
          A = Q /(U*LMDT) 
          Areas.append(A)
          U_q.append(U)
        else:
          y = Temp[cold][-1]
          x = Temp[names[k]][-1]
          Temp[cold].append(y)
          Temp[names[k]].append(x)       
    idx += 1

  cp_hot = Prop['hot'][0] # cp
  dt = Util['cold'][1]-Util['cold'][0]
  cp_cold = Prop['cold'][0]*dt 

  for word in names:
    x0 = Temp[word][-1]
    x = Target[word][1]
    cp = Prop[word][0]
    Q = cp*abs(x-x0)
    
    if x0 < x : # cooling activate
      flow = Q/cp_cold
      U = 1.0 / ( 1.0/Prop[word][1]+ 1.0/Prop['cold'][1])
      y0 = Util['cold'][0]
      y = Util['cold'][1]
      d1 = x0- y
      d2 = x - y0   
      if d2 != 0 and d1*d2 > 0:
        check = 'ok'
      else:
        check = 'no'
      

      # so troca calor se a temperatura fria final for menor q a temperatura quente final
      if x0 > y and x > y0 and check == 'ok': 
        #print('d1,d2',d1,d2)
        LMDT = (d1 - d2) / np.log(d1/d2)
        A = Q /(U*LMDT) # convert MW to kW
        kW = Q
        Gasta_cold.append([A,flow,kW])   
    else:
      flow = Q / cp_hot 
      U = 1.0 / ( 1.0/Prop[word][1]+ 1.0/Prop['cold'][1])
      y0 = Util['hot'][0]
      y = Util['hot'][1]
      d1 = y-x0 
      d2 = y0  -x   
      if d2 != 0 and d1*d2 > 0:
        check = 'ok'
      else:
        check = 'no'
      if x0 < y and x < y0 and check == 'ok': # so troca calor se a temperatura fria final for menor q a temperatura quente final
        #print('d1,d2',d1,d2)
        LMDT = (d1 - d2) / np.log(d1/d2)
        A = Q /(U*LMDT)
        kW= Q
        Gasta_hot.append([A,flow,kW])   

  Cost = 0
  #a, n, h, c = 4333,0.6, 150.53, 53.064 # L.V. Pavao et al. / Energy 142 (2018) 356e372
  a1,a2, h, c, N_EXP = 8000, 500, 80, 10, 0.75 # Chemical Engineering Science 158 (2017) 96–107- Pavao et al

  for A in Areas:
    Cost += a1 + a2*A**N_EXP #a*A**n
  for x in Gasta_hot:
    Cost += a1+a2*A**N_EXP #a*A**n
    Cost += h*x[2] # x[2]: steam 

  for x in Gasta_cold:
    Cost += a1+a2*A**N_EXP # a*x[0]**n
    Cost += c*x[2] # x[2]: cold water kW
  if np.isnan(Cost)==True:
    Cost = 1e99
  if ans == False:
    return Cost
  else:
    return Cost, matrix_trocas, Temp

In [ ]:
##w = [0.3]*4+[0.8]*4+[0.5]*4+[1]*12
#print(Cost_find(w))

In [ ]:
function = Cost_find


In [ ]:
def BestSource(GlobMin,GlobPars,Foods):
  FoodNumber=len(Foods[:,0])
  D=len(Foods[0,:])
  solution=np.zeros(D)
  for i in range(FoodNumber):
    if (f[i]<GlobMin):
      GlobMin=f[i]
      for j in range(D):
        GlobPars[j]=Foods[i,j]
  return GlobMin,GlobPars,Foods

In [ ]:
def init(index,Foods,trial,f,fitness,MIN,MAX):
  D=len(Foods[0,:])
  FoodNumber=len(Foods[:,0])
  solution=np.zeros(D)
  for j in range(D):
    r=np.random.random()
    Foods[index,j]=r*(MAX[j]-MIN[j])+MIN[j]
    solution[j]=Foods[index,j]
    
  f[index]=function(solution)
  fitness[index]=CalcFit(f[index])
  trial[index]=0
  
  return Foods,trial,f,fitness

In [ ]:
def initial(fitness,trial,f,Foods,GlobMin,GlobPars,MIN,MAX):
  D=len(Foods[0,:])
  FoodNumber=len(Foods[:,0])
  for i in range(FoodNumber):
    Foods,trial,f,fitness=init(i,Foods,trial,f,fitness,MIN,MAX) 
  GlobMin=f[0]
  for i in range(D):
    GlobPars[i]=Foods[0,i]
  return  f,Foods,GlobMin,GlobPars


In [ ]:
def EmployedBees(trial,Foods,MIN,MAX):
  FoodNumber=len(Foods[:,0])
  NP=FoodNumber
  D=len(Foods[0,:])
  solution=np.zeros(D)
  for i in range(FoodNumber):
    r = np.random.random()
    par2chan= int(r*D)
    r = np.random.random()
    neighbour=int(r*D)
    if(neighbour >= NP):
      neighbour=NP-1
      
    while(neighbour==i):
      r = np.random.random()
      neighbour=int(r*FoodNumber)
      if(neighbour >= NP):
        neighbour=NP-1
    for j in range(D):
      solution[j]=Foods[i,j]

    r = np.random.random()
    A=Foods[i,par2chan]
    B=Foods[neighbour,par2chan]
    solution[par2chan]=Foods[i,par2chan]+(A-B)*(r-0.5)*2;
    if (solution[par2chan]<MIN[par2chan]):
      solution[par2chan]=MIN[par2chan]
    if (solution[par2chan]>MAX[par2chan]):
      solution[par2chan]=MAX[par2chan]
    ObjValSol=function(solution)
    FitnessSol=CalcFit(ObjValSol)
    
    if(FitnessSol>fitness[i]):
      trial[i]=0
      for j in range(D):
        Foods[i,j]=solution[j]
      f[i]=ObjValSol
      fitness[i]=FitnessSol
    else:
      trial[i]=trial[i]+1 
      
  return trial,Foods

In [ ]:
def CalcProb(fitness,prob):
  FoodNumber=len(fitness)
  maxfit=fitness[0]
  for i in range(1,FoodNumber):    
    if (fitness[i]>maxfit):
      maxfit=fitness[i]
  for i in range(FoodNumber):
    prob[i]=(0.9*(fitness[i]/maxfit))+0.1
  return fitness,prob


In [ ]:
def OnlookerBees(trial,Foods,MIN,MAX):
  D=len(Foods[0,:])
  solution=np.zeros(D)
  FoodNumber=len(Foods[:,0])
  i=0
  t=0
  while(t<FoodNumber):
    r = np.random.random()
    if(r<prob[i]): 
      r = np.random.random()
      t=t+1
      par2chan=int(r*D)
      r = np.random.random()
      neighbour=int(r*FoodNumber)

      while(neighbour==i):
        r = np.random.random()
        neighbour=int(r*FoodNumber)
      for j in range(D):
        solution[j]=Foods[i,j]

      r = np.random.random()
      A=Foods[i,par2chan]
      B=Foods[neighbour,par2chan]
      solution[par2chan]=A+(A-B)*(r-0.5)*2

      if (solution[par2chan]<MIN[par2chan]):
              solution[par2chan]=MIN[par2chan]
      if (solution[par2chan]>MAX[par2chan]):
              solution[par2chan]=MAX[par2chan]
      ObjValSol=function(solution)
      FitnessSol=CalcFit(ObjValSol)
              
      if(FitnessSol>fitness[i]):
        trial[i]=0
        for j in range(D):
          Foods[i,j]=solution[j]
        f[i]=ObjValSol
        fitness[i]=FitnessSol
      else:
        trial[i]=trial[i]+1
      i=i+1
      if (i==FoodNumber):
        i=0
  return trial,Foods


In [ ]:
def ScoutBees(fitness,f,Foods,trial,MIN,MAX):
  FoodNumber=len(Foods[:,0])
  max_trial=0
  for i in range(1,FoodNumber):
    if (trial[i]>trial[max_trial]):
      max_trial=i
  if(trial[max_trial]>=limit):
    Foods,trial,f,fitness=init(max_trial,Foods,trial,f,fitness,MIN,MAX)
  return trial,Foods


In [ ]:
'''/*Main program of the ABC algorithm*/'''

NPAR=30 #Numero de fontes de comida
ITE=100 #ITERACOES (maxCycle)
PAR=n_hot*n_trocas* 2 #NUM DE PARAMETROS A SER OTIMIZADOS
MAX=[] # MAXIMO DE CADA PARAMETRO
MIN=[] # MINIMO DE CADA PARAMETRO
for k in range(PAR):
  MAX.append(1)
  MIN.append(0)

limit=100 # quantas vezes obtem resposta identica antes de encerrar 
runtime=4 # quantas vezes vai rodar para tirar a media

f=np.zeros(NPAR) 
Foods=np.zeros((NPAR,PAR)) 
solution=np.zeros(PAR)
fitness=np.zeros(NPAR)
trial=np.zeros(NPAR)
prob=np.zeros(NPAR)
GlobPars=np.zeros(PAR)
GlobMins=np.zeros(runtime)
GlobMin=1e99
BestAll = 1e99
for run in range(runtime):
  f,Foods,GlobMin,GlobPars= initial(fitness,trial,f,Foods,GlobMin,GlobPars,MIN,MAX)
  GlobMin,GlobPars,Foods=BestSource(GlobMin,GlobPars,Foods)

  for iter in range(ITE):
    trial,Foods=EmployedBees(trial,Foods,MIN,MAX)
    fitness,prob=CalcProb(fitness,prob)
    trial,Foods=OnlookerBees(trial,Foods,MIN,MAX)
    GlobMin,GlobPars,Foods=BestSource(GlobMin,GlobPars,Foods)
    trial,Foods=ScoutBees(fitness,f,Foods,trial,MIN,MAX)
    if BestAll > GlobMin:
      x_best = GlobPars
      cost_best = GlobMin




  print("run",run+1,"GBest=",GlobMin," \n");
  GlobMins[run]=GlobMin

mean=np.average(GlobMins)


print("Means of",runtime,"runs:",mean,"\n")

<ipython-input-4-88099c6ab93d>:85: RuntimeWarning: invalid value encountered in double_scalars
  LMDT = (d1 - d2) / np.log(d1/d2)
<ipython-input-4-88099c6ab93d>:139: RuntimeWarning: invalid value encountered in double_scalars
  LMDT = (d1 - d2) / np.log(d1/d2)


run 1 GBest= 2097159.249217199  

run 2 GBest= 1907501.87512557  

run 3 GBest= 1835769.6064568174  

run 4 GBest= 1929171.8100951668  

Means of 4 runs: 1942400.6352236883 



In [ ]:
PAR

48

In [ ]:
Cost, Conditions, Temp =Cost_find(x_best,ans = True)
print('Cost',Cost)
print('Conditions',Conditions)
print('Temp',Temp)
print('GlobPars',GlobPars)

Cost 1929171.8100951668
Conditions [['C7', 'C2', 'C1'], ['C4', 'C6', 'C5'], ['C1', 'C6', 'C6'], ['C6', 'C3', 'C6'], ['C7', 'C1', 'C6'], ['C7', 'No', 'C5'], ['C6', 'C3', 'C4'], ['C5', 'C3', 'C4']]
Temp {'H1': [180, 180, 110.7829950194745, 110.7829950194745], 'H2': [280, 280, 280, 120.0], 'H3': [180, 88.79009108402751, 88.79009108402751, 88.79009108402751], 'H4': [140, 123.67508376684654, 51.729084006823086, 51.729084006823086], 'H5': [220, 213.09790999837375, 213.09790999837375, 173.66701332913357], 'H6': [180, 180, 180, 180], 'H7': [200, 127.37024691182036, 110.33630686336473, 110.33630686336473], 'H8': [120, 93.55820571964696, 93.55820571964696, 89.52982035414828], 'C1': [40, 176.81486337395876, 176.81486337395876, 176.81486337395876], 'C2': [100, 134.60850249026274], 'C3': [40, 101.66799979430581, 116.26851983583921, 116.26851983583921], 'C4': [50, 50, 50, 63.42795121832892], 'C5': [50, 94.06965713392172, 254.06965713392174, 254.06965713392174], 'C6': [90, 99.79494973989208, 143.3728

<ipython-input-4-88099c6ab93d>:139: RuntimeWarning: invalid value encountered in double_scalars
  LMDT = (d1 - d2) / np.log(d1/d2)
